# Dependencias

In [1]:
from pymongo import MongoClient
import os
import copy

class MongoWrapper:

    def __init__(self):

        self.client = MongoClient(
            'mongodb://' + os.getenv('MONGO_USERNAME') + ':' + os.getenv('MONGO_PASSWORD') + \
                '@' + os.getenv('MONGO', 'localhost:60222')
        )
        self.db = self.client[os.getenv('MONGO_BBDD', 'admin')]

    def set_one(self, collection, document):

        return self.db[collection].insert_one(document).acknowledged

    def get_one(self, collection, query):

        return self.db[collection].find_one(query)

    def get_many(self, collection, query):

        return self.db[collection].find(query)

    def __upsert(self, collection, query, document, options):

        return self.db[collection].update(query, document, **options).get('ok') == 1.0

    def upsert_one(self, collection, query, document):

        return self.__upsert(collection, query, document, {"upsert": True, "multi": False})

    def upsert_many(self, collection, query, document):

        return self.__upsert(collection, query, document, {"upsert": True, "multi": True})

    def delete_one(self, collection, query):

        self.db[collection].delete_one(query)

    def delete_many(self, collection, query):

        self.db[collection].delete_many(query)

# Recopilación de información de distintos canales

In [6]:
MONGO_CLIENT = MongoWrapper()
USERS_COLLECTION = 'users'
DATA_COLLECTION = 'nodered'
MQTT_COLLECTION = 'mqtt'
TWTITTER_COLLECTION = 'twitter'

In [10]:
usernames = {user['user']: user['id'] for user in [copy.deepcopy(user) for user in MONGO_CLIENT.get_many(USERS_COLLECTION, {})]}

In [11]:
usernames

{'CALZADOS M&N': 265790648, 'LOS AMIGOS': -1, "LETICIA'S": -2}

In [23]:
id_user ={'id': usernames['CALZADOS M&N'], 'user': 'CALZADOS M&N'}

In [24]:
id_user

{'id': 265790648, 'user': 'CALZADOS M&N'}

In [25]:
for item in MONGO_CLIENT.get_many(MQTT_COLLECTION, {'mqttId': id_user['id']}):
    print(item)

{'_id': ObjectId('5fdb928f4ed3af001076a8b0'), 'mqttId': 265790648, 'content': {'ambiance': 22, 'ocupation': 3}, 'date': 1608225423}


In [30]:
for item in MONGO_CLIENT.get_many(TWTITTER_COLLECTION, {'tweetId': id_user['user'].replace(' ', '_')}):
    print(item)

{'_id': ObjectId('5fdb9d004ed3af001076a8b1'), 'content': '#fba_iot_tfm_2020_bot #CALZADOS_M&N Tax efficiency makes a big difference. Let’s talk before year-end.\n https://t.co/LVVsG1lVmi', 'tweetId': 'CALZADOS_M&N', 'show': True, 'date': 1608228096}


In [28]:
id_user['user'].replace(' ', '_')

'CALZADOS_M&N'